<a href="https://colab.research.google.com/github/RyosukeHanaoka/TechTeacher/blob/main/Finetuned_Mediapipe_Object_Detector(2023_09_28).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Project: /mediapipe/_project.yaml
Book: /mediapipe/_book.yaml

<link rel="stylesheet" href="/mediapipe/site.css">

# Object detection model customization guide

<table align="left" class="buttons">
  <td>
    <a href="https://colab.research.google.com/github/googlesamples/mediapipe/blob/main/examples/customization/object_detector.ipynb" target="_blank">
      <img src="https://developers.google.com/static/mediapipe/solutions/customization/colab-logo-32px_1920.png" alt="Colab logo"> Run in Colab
    </a>
  </td>

  <td>
    <a href="https://github.com/googlesamples/mediapipe/blob/main/examples/customization/object_detector.ipynb" target="_blank">
      <img src="https://developers.google.com/static/mediapipe/solutions/customization/github-logo-32px_1920.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
</table>

MediaPipe オブジェクト検出ソリューションには、アプリケーションの機械学習 (ML) にすぐに使用できるモデルがいくつか用意されています。 ただし、提供されたモデルでカバーされていないオブジェクトを検出する必要がある場合は、独自のデータと MediaPipe Model Maker を使用して提供されたモデルのいずれかをカスタマイズできます。 このモデル変更ツールは、提供されたデータを使用してモデルを再構築します。 この方法は、新しいモデルをトレーニングするよりも速く、特定のアプリケーションにとってより役立つモデルを生成できます。

次のセクションでは、Model Maker を使用して、独自のデータを使用してオブジェクト検出用の事前構築モデルを再トレーニングし、その後 MediaPipe Object Detector で使用できるようにする方法を示します。 この例では、画像内の Android フィギュアを検出するために汎用オブジェクト検出モデルを再トレーニングします。

## 設定

このセクションでは、モデルを再トレーニングするために開発環境をセットアップするための主要な手順について説明します。 これらの手順では、[Google Colab](https://colab.research.google.com/) を使用してモデルを更新する方法について説明します。また、独自の開発環境で Python を使用することもできます。 MediaPipe を使用するための開発環境のセットアップに関する一般情報 (プラットフォームのバージョン要件など) については、[Python のセットアップ ガイド](https://developers.google.com/mediapipe/solutions/setup_python) を参照してください。

**注意:** この MediaPipe ソリューション プレビューは初期リリースです。 [詳細](https://developers.google.com/mediapipe/solutions/about)。

モデルをカスタマイズするためのライブラリをインストールするには、次のコマンドを実行します。

In [1]:
!python --version
!pip install --upgrade pip
!pip install mediapipe-model-maker

Python 3.10.12
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
  Obtaining dependency information for mediapipe-model-maker from https://files.pythonhosted.org/packages/9a/f4/26b03c5b5cacd9bcbaf06a906915f56d6b7cd7276c28c246db4a42c1f35b/mediapipe_model_maker-0.2.1.3-py3-none-any.whl.metadata
  Obtaining dependency information for mediapipe>=0.10.0 from https://files.pythonhosted.org/packages/4a/58/1ec1516ee0f3e9f8073bf72a4c1653712b737068584a8cc91d1906d13a10/mediapipe-0.10.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for tensorflow-addons from https://files.pythonhosted.org/packages/12/34/221b939127e629c1ebbfb8a0281f67b084515ed53e5042582238817bcc54/tensorflow_addons-0.21.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining depend

次のコードを使用して、必要な Python クラスをインポートします。

Use the following code to import the required Python classes:

In [2]:
from google.colab import files
import os
import json
import tensorflow as tf
assert tf.__version__.startswith('2')

from mediapipe_model_maker import object_detector

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!wget -q -O efficientdet.tflite -q https://storage.googleapis.com/mediapipe-models/object_detector/efficientdet_lite0/int8/1/efficientdet_lite0.tflite

## データの準備

オブジェクト検出用にモデルを再トレーニングするには、完成したモデルで識別できるようにするアイテムまたはクラスを含むデータセットが必要です。 これを行うには、パブリック データセットをトリミングしてユースケースに関連するクラスのみにするか、独自のデータセットをコンパイルするか、またはその両方を組み合わせます。データセットは、新しいモデルのトレーニングに必要なデータセットよりも大幅に小さくなる可能性があります。 たとえば、多くの参照モデルのトレーニングに使用される COCO データセットには、91 クラスのオブジェクトを含む数十万の画像が含まれています。 Model Maker を使用した転移学習は、推論精度の目標に応じて、より小さなデータセットを使用して既存のモデルを再トレーニングしても、良好なパフォーマンスを維持できます。 これらの手順では、2 種類のアンドロイド フィギュア、つまり 2 つのクラス、合計 62 個のトレーニング画像を含む小さなデータセットを使用します。

サンプル データセットをダウンロードするには、次のコードを使用します。

In [5]:
train_dataset_path = "/content/drive/MyDrive/xpFilesPNG/train/"
validation_dataset_path = "/content/drive/MyDrive/xpFilesPNG/val"

### サポートされているデータセット形式
Model Maker Object Detection API は、次のデータセット形式の読み取りをサポートしています。

#### COCO 形式

COCO データセット形式には、すべての画像を保存する「data」ディレクトリと、すべての画像のオブジェクト注釈を含む単一の「labels.json」ファイルがあります。

### データセットを確認する

「labels.json」ファイルからカテゴリを出力して、データセットの内容を確認します。 カテゴリは合計 3 つあるはずです。 インデックス 0 は常に、データセット内で使用されない可能性がある「バックグラウンド」クラスに設定されます。 `android` と `pig_android` という 2 つの非バックグラウンド カテゴリがあるはずです。

In [6]:
with open(os.path.join(train_dataset_path, "labels.json"), "r") as f:
  labels_json = json.load(f)
for category_item in labels_json["categories"]:
  print(f"{category_item['id']}: {category_item['name']}")

1: MCP
2: PIP
3: DIP
4: IP
5: Wrist
6: MCP1st


データセットをより深く理解するには、いくつかのサンプル画像をその境界ボックスとともにプロットします。

In [7]:
#@title トレーニングデータの視覚化
import matplotlib.pyplot as plt
from matplotlib import patches, text, patheffects
from collections import defaultdict
import math

def draw_outline(obj):
  obj.set_path_effects([patheffects.Stroke(linewidth=4,  foreground='black'), patheffects.Normal()])
def draw_box(ax, bb):
  patch = ax.add_patch(patches.Rectangle((bb[0],bb[1]), bb[2], bb[3], fill=False, edgecolor='red', lw=2))
  draw_outline(patch)
def draw_text(ax, bb, txt, disp):
  text = ax.text(bb[0],(bb[1]-disp),txt,verticalalignment='top'
  ,color='white',fontsize=10,weight='bold')
  draw_outline(text)
def draw_bbox(ax, annotations_list, id_to_label, image_shape):
  for annotation in annotations_list:
    cat_id = annotation["category_id"]
    bbox = annotation["bbox"]
    draw_box(ax, bbox)
    draw_text(ax, bbox, id_to_label[cat_id], image_shape[0] * 0.05)
def visualize(dataset_folder, max_examples=None):
  with open(os.path.join(dataset_folder, "labels.json"), "r") as f:
    labels_json = json.load(f)
  images = labels_json["images"]
  cat_id_to_label = {item["id"]:item["name"] for item in labels_json["categories"]}
  image_annots = defaultdict(list)
  for annotation_obj in labels_json["annotations"]:
    image_id = annotation_obj["image_id"]
    image_annots[image_id].append(annotation_obj)

  if max_examples is None:
    max_examples = len(image_annots.items())
  n_rows = math.ceil(max_examples / 3)
  fig, axs = plt.subplots(n_rows, 3, figsize=(24, n_rows*8)) # 3 columns(2nd index), 8x8 for each image
  for ind, (image_id, annotations_list) in enumerate(list(image_annots.items())[:max_examples]):
    ax = axs[ind//3, ind%3]
    img = plt.imread(os.path.join(dataset_folder, "images", images[image_id]["file_name"]))
    ax.imshow(img)
    draw_bbox(ax, annotations_list, cat_id_to_label, img.shape)
  plt.show()

visualize(train_dataset_path, 9)

### データセットを作成する

Dataset クラスには、COCO または PASCAL VOC データセットをロードするための 2 つのメソッドがあります。
* `Dataset.from_coco_folder`
* `Dataset.from_pascal_voc_folder`

android_figurines データセットは COCO データセット形式であるため、`from_coco_folder` メソッドを使用して、`train_dataset_path` と `validation_dataset_path` にあるデータセットを読み込みます。 データセットをロードすると、データは指定されたパスから解析され、標準化された [TFRecord](https://www.tensorflow.org/tutorials/load_data/tfrecord) 形式に変換され、後で使用するためにキャッシュされます。 同じデータセットの複数のキャッシュを保存しないようにするには、`cache_dir` の場所を作成し、すべてのトレーニングでそれを再利用する必要があります。

In [8]:
train_data = object_detector.Dataset.from_coco_folder(train_dataset_path, cache_dir="/tmp/od_data/train")
validation_data = object_detector.Dataset.from_coco_folder(validation_dataset_path, cache_dir="/tmp/od_data/validation")
print("train_data size: ", train_data.size)
print("validation_data size: ", validation_data.size)

train_data size:  411
validation_data size:  46


## 再学習過程

データの準備が完了したら、トレーニングデータで定義した新たな物体やクラスを認識するように再学習過程を開始することができる。以下の手順では、前のセクションで準備したデータを使用して画像分類モデルを再トレーニングし、2種類のアンドロイド フィギュアを認識する。

### Set retraining options

再トレーニングを実行するには、トレーニング データセットのほかに、モデルの出力ディレクトリとモデル アーキテクチャといういくつかの必要な設定があります。 HParams を使用して、出力ディレクトリの export_dir パラメータを指定します。 SupportedModels クラスを使用してモデル アーキテクチャを指定します。 物体検出ソリューションは、次のモデル アーキテクチャをサポートします。
* `MobileNet-V2`
* `MobileNet-MultiHW-AVG`
トレーニング パラメーターのより高度なカスタマイズについては、以下の [ハイパーパラメーター](#hyperparameters) セクションをご覧ください。

必要なパラメータを設定するには、次のコードを使用します。

In [16]:
spec = object_detector.SupportedModels.MOBILENET_MULTI_AVG
#hparams = object_detector.HParams(export_dir='exported_model')
hparams = object_detector.HParams(
    learning_rate=0.3,
    batch_size=8,
    epochs=100,
    export_dir='exported_model'
    )
options = object_detector.ObjectDetectorOptions(
    supported_model=spec,
    hparams=hparams
)

### 再学習過程


In [17]:
model = object_detector.ObjectDetector.create(
    train_data=train_data,
    validation_data=validation_data,
    options=options)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/functional.py:639: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


Using existing files at /tmp/model_maker/object_detector/mobilenetmultiavg
Model: "retina_net_model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobile_net_1 (MobileNet)    {'2': (None, 64, 64, 32   3704416   
                             ),                                  
                              '3': (None, 32, 32, 64             
                             ),                                  
                              '4': (None, 16, 16, 16             
                             0),                                 
                              '5': (None, 8, 8, 192)             
                             , '6': (None, 1, 1, 128             
                             0)}                                 
                                                                 
 fpn_1 (FPN)                 {'5': (None, 8, 8, 128)   144928    
                             , '4': (No

/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:452: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn(


51/51 [==============================] - 53s 267ms/step - total_loss: 3.0252 - cls_loss: 2.2744 - box_loss: 0.0138 - model_loss: 2.9623 - val_total_loss: 2.0853 - val_cls_loss: 1.2386 - val_box_loss: 0.0157 - val_model_loss: 2.0224
Epoch 2/100
 2/51 [>.............................] - ETA: 8s - total_loss: 1.8390 - cls_loss: 1.1943 - box_loss: 0.0116 - model_loss: 1.7762 

KeyboardInterrupt: ignored

### モデルのパフォーマンスを評価する

モデルをトレーニングした後、検証データセットを用いてモデルを評価し、損失率と coco_metrics を出力します。 モデルのパフォーマンスを評価するための最も重要な指標は、通常、平均精度の「AP」ココメトリックです。

In [12]:
loss, coco_metrics = model.evaluate(validation_data, batch_size=4)
print(f"Validation loss: {loss}")
print(f"Validation coco metrics: {coco_metrics}")

12/12 [==============================] - 3s 93ms/step - total_loss: 0.2276 - cls_loss: 0.1210 - box_loss: 9.0608e-04 - model_loss: 0.1663
creating index...
index created!
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.58s).
Accumulating evaluation results...
DONE (t=0.09s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.648
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.981
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.732
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.448
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.677
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.242
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.709
 Average Recall     (AR)

##Boundingboxの境界線に沿って、関節部分の画像をクロップする。

In [13]:
# 必要なライブラリのインポート
from PIL import Image
import os
from PIL import ImageDraw

base_dir = "/content/drive/MyDrive/xpFile_bbox"
if not os.path.exists(base_dir):
    os.mkdir(base_dir)

# bboxをクロップして保存する関数
def save_bbox_image(image_item, labels_json):
    # ファイル名から必要な部分を取得
    file_name = image_item["file_name"]
    dir_name = file_name[:6] + "_" + file_name[43:49]
    dir_path = os.path.join(base_dir, dir_name)

    if not os.path.exists(dir_path):
        os.mkdir(dir_path)

    classes = ['MCP', 'PIP', 'DIP', 'IP', 'Wrist', 'MCP1st']
    for class_name in classes:
        class_dir = os.path.join(dir_path, class_name)
        if not os.path.exists(class_dir):
            os.mkdir(class_dir)

    ann_image_dir = os.path.join(dir_path, "annotationImage")
    if not os.path.exists(ann_image_dir):
        os.mkdir(ann_image_dir)

    image_path = os.path.join(train_dataset_path, "images", file_name)
    with Image.open(image_path) as img:
        drawed_img = img.copy()
        draw = ImageDraw.Draw(drawed_img)

        for annotation_item in labels_json["annotations"]:
            if annotation_item["image_id"] == image_item["id"]:
                class_id = annotation_item["category_id"]
                class_name = [category["name"] for category in labels_json["categories"] if category["id"] == class_id][0]
                bbox = annotation_item["bbox"]

                cropped_img = img.crop((bbox[0], bbox[1], bbox[0] + bbox[2], bbox[1] + bbox[3]))
                cropped_img_name = f"{dir_name}_{class_name}_{annotation_item['id']}.png"
                cropped_img_path = os.path.join(dir_path, class_name, cropped_img_name)
                cropped_img.save(cropped_img_path)

                # bboxを描画
                draw.rectangle([bbox[0], bbox[1], bbox[0] + bbox[2], bbox[1] + bbox[3]], outline='red', width=2)

        ann_image_name = f"{dir_name}_annotation.png"
        ann_image_path = os.path.join(ann_image_dir, ann_image_name)
        drawed_img.save(ann_image_path)

# 各画像に対して関数を実行
for image_item in labels_json["images"]:
    save_bbox_image(image_item, labels_json)

print("All bbox images saved!")

All bbox images saved!


## モデルのエクスポート

モデルを作成した後、後でデバイス上のアプリケーションで使用できるように、モデルを Tensorflow Lite モデル形式に変換してエクスポートします。 エクスポートには、ラベルマップを含むモデルメタデータも含まれます。

In [15]:
model.export_model()
!ls exported_model
files.download('exported_model/model.tflite')

Exporting a floating point model


/usr/local/lib/python3.10/dist-packages/keras/src/engine/functional.py:639: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)
/usr/local/lib/python3.10/dist-packages/keras/src/engine/functional.py:639: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


ls: cannot access 'exported_model': No such file or directory


FileNotFoundError: ignored

## モデルの量子化

モデルを作成した後、後でデバイス上のアプリケーションで使用できるように、モデルを Tensorflow Lite モデル形式に変換してエクスポートします。 エクスポートにはモデルも含まれます。 モデル量子化は、比較的わずかな精度の低下のみでモデル サイズを削減し、予測速度を向上させることができるモデル変更手法です。

ガイドのこのセクションでは、モデルに量子化を適用する方法について説明します。 Model Maker は、オブジェクト検出器の 2 つの形式の量子化をサポートしています。
1. 量子化対応トレーニング: CPU 使用のための 8 ビット整数精度
2. トレーニング後の量子化: GPU使用のための16 ビット浮動小数点精度。

Model quantization is a model modification technique that can reduce the model size and improve the speed of predictions with only a relatively minor decrease in accuracy.

This section of the guide explains how to apply quantization to your model. Model Maker supports two forms of quantization for object detector:
1. Quantization Aware Training: 8 bit integer precision for CPU usage
2. Post-Training Quantization: 16 bit floating point precision for GPU usage

### 量子化を意識したトレーニング (int8 量子化)
量子化対応トレーニング (QAT) は、モデルを完全にトレーニングした後に行われる微調整ステップです。 この手法では、8 ビット整数量子化の精度の低さを考慮して、推論時間量子化をエミュレートするモデルをさらに調整します。 標準 CPU を備えたオンデバイス アプリケーションの場合は、Int8 精度を使用します。 詳細については、[TensorFlow Lite](https://www.tensorflow.org/model_optimization/guide/quantization/training) ドキュメントを参照してください。

量子化対応トレーニングを適用して int8 モデルにエクスポートするには、`QATHParams` 設定を作成し、`quantization_aware_training` メソッドを実行します。 `QATHParams` の詳細な使用法については、以下の **ハイパーパラメータ** セクションを参照してください。

### Quantization aware training (int8 quantization)
Quantization aware training (QAT) is a fine-tuning step which happens after fully training your model. This technique further tunes a model which emulates inference time quantization in order to account for the lower precision of 8 bit integer quantization. For on-device applications with a standard CPU, use Int8 precision. For more information, see the [TensorFlow Lite](https://www.tensorflow.org/model_optimization/guide/quantization/training) documentation.

To apply quantization aware training and export to an int8 model, create a `QATHParams` configuration and run the `quantization_aware_training` method. See the **Hyperparameters** section below on detailed usage of `QATHParams`.

In [ ]:
qat_hparams = object_detector.QATHParams(learning_rate=0.3, batch_size=4, epochs=10, decay_steps=6, decay_rate=0.96)
model.quantization_aware_training(train_data, validation_data, qat_hparams=qat_hparams)
qat_loss, qat_coco_metrics = model.evaluate(validation_data)
print(f"QAT validation loss: {qat_loss}")
print(f"QAT validation coco metrics: {qat_coco_metrics}")

QAT ステップでは、トレーニングのパラメーターを調整するために複数の実行が必要になることがよくあります。 `create` メソッドを使用してモデル トレーニングを再実行する必要がないようにするには、QAT を再度実行するために、`restore_float_ckpt` メソッドを使用してモデルの状態を完全にトレーニングされた浮動小数点モデルに戻します (`create` メソッドの実行後)。

The QAT step often requires multiple runs to tune the parameters of training. To avoid having to rerun model training using the `create` method, use the `restore_float_ckpt` method to restore the model state back to the fully trained float model(After running the `create` method) in order to run QAT again.

In [ ]:
new_qat_hparams = object_detector.QATHParams(learning_rate=0.9, batch_size=4, epochs=15, decay_steps=5, decay_rate=0.96)
model.restore_float_ckpt()
model.quantization_aware_training(train_data, validation_data, qat_hparams=new_qat_hparams)
qat_loss, qat_coco_metrics = model.evaluate(validation_data)
print(f"QAT validation loss: {qat_loss}")
print(f"QAT validation coco metrics: {qat_coco_metrics}")

最後に、export_model を使用して int8 量子化モデルにエクスポートします。 export_model 関数は、quantization_aware_training が実行されたかどうかに応じて、float32 モデルまたは int8 モデルのいずれかに自動的にエクスポートします。

Finally, us the `export_model` to export to an int8 quantized model. The `export_model` function will automatically export to either float32 or int8 model depending on whether `quantization_aware_training` was run.

In [ ]:
model.export_model('model_int8_qat.tflite')
!ls -lh exported_model
files.download('exported_model/model_int8_qat.tflite')

### トレーニング後の量子化 (fp16 量子化)

トレーニング後のモデルの量子化は、比較的わずかな精度の低下のみでモデルのサイズを削減し、予測速度を向上させることができるモデル変更手法です。 このアプローチでは、たとえば 32 ビット浮動小数点数を 16 ビット浮動小数点数に変換することにより、モデルによって処理されるデータのサイズが削減されます。 GPU の使用には Float16 量子化が推奨されます。 詳細については、[TensorFlow Lite](https://www.tensorflow.org/model_optimization/guide/quantization/post_training) ドキュメントを参照してください。

まず、MediaPipe Model Maker 量子化モジュールをインポートします。

### Post-training quantization (fp16 quantization)

Post-training model quantization is a model modification technique that can reduce the model size and improve the speed of predictions with only a relatively minor decrease in accuracy. This approach reduces the size of the data processed by the model, for example by transforming 32-bit floating point numbers to 16-bit floats. Float16 quantization is reccomended for GPU usage. For more information, see the [TensorFlow Lite](https://www.tensorflow.org/model_optimization/guide/quantization/post_training) documentation.

First, import the MediaPipe Model Maker quantization module:

In [ ]:
from mediapipe_model_maker import quantization

for_float16() クラス メソッドを使用して QuantizationConfig オブジェクトを定義します。 この構成は、32 ビット浮動小数点数の代わりに 16 ビット浮動小数点数を使用するようにトレーニングされたモデルを変更します。 QuantizationConfig クラスの追加パラメーターを設定することで、量子化プロセスをさらにカスタマイズできます。

Define a QuantizationConfig object using the `for_float16()` class method. This configuration modifies a trained model to use 16-bit floating point numbers instead of 32-bit floating point numbers. You can further customize the quantization process by setting additional parameters for the QuantizationConfig class.

In [ ]:
quantization_config = quantization.QuantizationConfig.for_float16()

追加の quantization_config オブジェクトを使用してモデルをエクスポートし、トレーニング後の量子化を適用します。 以前に quantization_aware_training を実行したことがある場合は、まず、restore_float_ckpt を使用してモデルを float モデルに変換し直す必要があることに注意してください。

Export the model using the additional quantization_config object to apply post-training quantization. Note that if you previously ran `quantization_aware_training`, you must first convert the model back to a float model by using `restore_float_ckpt`.

In [ ]:
model.restore_float_ckpt()
model.export_model(model_name="model_fp16.tflite", quantization_config=quantization_config)
!ls -lh exported_model
files.download('exported_model/model_fp16.tflite')

##ハイパーパラメータ
ObjectDetectorOptions クラスを使用してモデルをさらにカスタマイズできます。このクラスには、SupportedModels、ModelOptions、および HParams の 3 つのパラメーターがあります。

SupportedModels enum クラスを使用して、トレーニングに使用するモデル アーキテクチャを指定します。 次のモデル アーキテクチャがサポートされています。

モバイルネット_V2
モバイルネット_V2_I320
MOBILENET_MULTI_AVG
MOBILENET_MULTI_AVG_I384
HParams クラスを使用して、モデルのトレーニングと保存に関連する他のパラメーターをカスタマイズします。

learning_rate: 勾配降下トレーニングに使用する学習率。 デフォルトは 0.3 です。
batch_size: トレーニングのバッチ サイズ。 デフォルトは 8 です。
エポック: データセットに対するトレーニング反復の数。 デフォルトは 30 です。
cosine_decay_epochs: コサイン減衰学習率のエポック数。 詳細については、「tf.keras.optimizers.schedules.CosineDecay」を参照してください。 デフォルトは None で、これはエポックに設定するのと同じです。
cosine_decay_alpha: コサイン減衰学習率のアルファ値。 詳細については、「tf.keras.optimizers.schedules.CosineDecay」を参照してください。 デフォルトは 1.0 で、コサイン減衰がないことを意味します。
ModelOptions クラスを使用して、モデル自体に関連するパラメーターをカスタマイズします。

l2_weight_decay: tf.keras.regulators.L2 で使用される L2 正則化ペナルティ。 デフォルトは 3e-5 です。
QATHParams クラスを使用して、量子化対応トレーニングのトレーニング パラメーターをカスタマイズします。

learning_rate: 勾配降下 QAT に使用する学習率。 デフォルトは 0.3 です。
batch_size: QAT のバッチ サイズ。 デフォルトは 8
エポック: データセットに対するトレーニング反復の数。 デフォルトは 15 です。
decay_steps: 指数関数的減衰の学習率減衰ステップ。 詳細については、「tf.keras.optimizers.schedules.ExponentialDecay」を参照してください。 デフォルトは 8
decay_rate: 指数関数的減衰の学習率減衰率。 詳細については、「tf.keras.optimizers.schedules.ExponentialDecay」を参照してください。 デフォルトは 0.96 です。

## Hyperparameters
You can further customize the model using the ObjectDetectorOptions class, which has three parameters for `SupportedModels`, `ModelOptions`, and `HParams`.

Use the `SupportedModels` enum class to specify the model architecture to use for training. The following model architectures are supported:
* MOBILENET_V2
* MOBILENET_V2_I320
* MOBILENET_MULTI_AVG
* MOBILENET_MULTI_AVG_I384

Use the `HParams` class to customize other parameters related to training and saving the model:
* `learning_rate`: Learning rate to use for gradient descent training. Defaults to 0.3.
* `batch_size`: Batch size for training. Defaults to 8.
* `epochs`: Number of training iterations over the dataset. Defaults to 30.
* `cosine_decay_epochs`: The number of epochs for cosine decay learning rate. See [tf.keras.optimizers.schedules.CosineDecay](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/schedules/CosineDecay) for more info. Defaults to None, which is equivalent to setting it to `epochs`.
* `cosine_decay_alpha`: The alpha value for cosine decay learning rate. See [tf.keras.optimizers.schedules.CosineDecay](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/schedules/CosineDecay) for more info. Defaults to 1.0, which means no cosine decay.

Use the `ModelOptions` class to customize parameters related to the model itself:
* `l2_weight_decay`: L2 regularization penalty used in [tf.keras.regularizers.L2](https://www.tensorflow.org/api_docs/python/tf/keras/regularizers/L2). Defaults to 3e-5.

Uset the `QATHParams` class to customize training parameters for Quantization Aware Training:
* `learning_rate`: Learning rate to use for gradient descent QAT. Defaults to 0.3.
* `batch_size`: Batch size for QAT. Defaults to 8
* `epochs`: Number of training iterations over the dataset. Defaults to 15.
* `decay_steps`: Learning rate decay steps for Exponential Decay. See [tf.keras.optimizers.schedules.ExponentialDecay](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/schedules/ExponentialDecay) for more information. Defaults to 8
* `decay_rate`: Learning rate decay rate for Exponential Decay. See [tf.keras.optimizers.schedules.ExponentialDecay](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/schedules/ExponentialDecay) for more information. Defaults to 0.96.

## ベンチマーク
以下は、サポートされているモデル アーキテクチャのベンチマーク結果の概要です。 これらのモデルは、このノートブックと同じ Android フィギュア データセットでトレーニングおよび評価されました。 モデルのベンチマーク結果を検討する際には、留意すべき重要な注意点がいくつかあります。
* Android フィギュア データセットは、62 個のトレーニング サンプルと 10 個の検証サンプルを含む小規模でシンプルなデータセットです。 データセットは非常に小さいため、トレーニング プロセスの差異によりメトリクスが大幅に変化する可能性があります。 このデータセットはデモ目的で提供されており、モデルのパフォーマンスを向上させるためにさらに多くのデータ サンプルを収集することをお勧めします。
* float32 モデルはデフォルトの HParams でトレーニングされ、int8 モデルの QAT ステップは `QATHParams(learning_rate=0.1,batch_size=4,epochs=30,decay_rate=1)` で実行されました。
* 独自のデータセットの場合、最良の結果を得るには、HParams と QATHParams の両方の値を調整する必要がある可能性があります。 トレーニング パラメーターの構成の詳細については、上記の [Hyperparameters](#hyperparameters) セクションを参照してください。
* すべての遅延数値は Pixel 6 でベンチマークされています。

## Benchmarking
Below is a summary of our benchmarking results for the supported model architectures. These models were trained and evaluated on the same android figurines dataset as this notebook. When considering the model benchmarking results, there are a few important caveats to keep in mind:
* The android figurines dataset is a small and simple dataset with 62 training examples and 10 validation examples. Since the dataset is quite small, metrics may vary drastically due to variances in the training process. This dataset was provided for demo purposes and it is recommended to collect more data samples for better performing models.
* The float32 models were trained with the default HParams, and the QAT step for the int8 models was run with `QATHParams(learning_rate=0.1, batch_size=4, epochs=30, decay_rate=1)`.
* For your own dataset, you will likely need to tune values for both HParams and QATHParams in order to achieve the best results. See the [Hyperparameters](#hyperparameters) section above for more information on configuring training parameters.
* All latency numbers are benchmarked on the Pixel 6.


<table>
<thead>
<col>
<col>
<colgroup span="2"></colgroup>
<colgroup span="2"></colgroup>
<colgroup span="2"></colgroup>
<tr>
<th rowspan="2">Model architecture</th>
<th rowspan="2">Input Image Size</th>
<th colspan="2" scope="colgroup">Test AP</th>
<th colspan="2" scope="colgroup">CPU Latency</th>
<th colspan="2" scope="colgroup">Model Size</th>
</tr>
<tr>
<th>float32</th>
<th>QAT int8</th>
<th>float32</th>
<th>QAT int8</th>
<th>float32</th>
<th>QAT int8</th>
</tr>
</thead>
<tbody>
<tr>
<td>MobileNetV2</td>
<td>256x256</td>
<td>88.4%</td>
<td>73.5%</td>
<td>48ms</td>
<td>16ms</td>
<td>11MB</td>
<td>3.2MB</td>
</tr>
<tr>
<td>MobileNetV2 I320</td>
<td>320x320</td>
<td>89.1%</td>
<td>75.5%</td>
<td>75ms</td>
<td>33.38ms</td>
<td>10MB</td>
<td>3.3MB</td>
</tr>
<tr>
<td>MobileNet MultiHW AVG</td>
<td>256x256</td>
<td>88.5%</td>
<td>70.0%</td>
<td>56ms</td>
<td>19ms</td>
<td>13MB</td>
<td>3.6MB</td>
</tr>
<tr>
<td>MobileNet MultiHW AVG I384</td>
<td>384x384</td>
<td>92.7%</td>
<td>73.4%</td>
<td>238ms</td>
<td>41ms</td>
<td>13MB</td>
<td>3.6MB</td>
</tr>

</tbody>
</table>

